데이터 확인

In [3]:
!unzip '/content/kor-eng.zip' -d koreng

Archive:  /content/kor-eng.zip
  inflating: koreng/_about.txt       
  inflating: koreng/kor.txt          


In [4]:
import string
new_lines = []
with open( '/content/koreng/kor.txt','r' ,encoding='utf-8' ) as f:
  lines = f.read().split('\n')
  for line in lines:
    try:
      txt = "".join( v for v in line if v not in string.punctuation).lower()
      txt = txt.split()
      txt = txt[0]+'\t'+txt[1]
      new_lines.append(txt)
    except IndexError as e:
      print(txt)


[]


In [5]:
new_lines[:5]

['go\t가', 'hi\t안녕', 'run\t뛰어', 'run\t뛰어', 'who\t누구']

학습용 데이터 만들기

In [6]:
import numpy as np
import torch
from torch.utils.data.dataset import Dataset

In [7]:
# 문장으로 부터 BOW
def get_BOW(corpus):
  BOW = {"<SOS>":0,"<EOS>":1}
  for line in corpus:
    for word in line.split():
      if word not in BOW.keys():
        BOW[word] = len(BOW.keys())
  return BOW

In [8]:
class Eng2Kor(Dataset):
  def __init__(self, filepath = '/content/koreng/kor.txt'):
    self.eng_corpus = [] # 영어 문장
    self.kor_corpus = [] # 한글 문장
    # 파일에서 데이터 읽음
    with open(filepath, 'r', encoding='utf-8') as f:
      lines = f.read().split('\n')
      for line in lines:
        try:
          # 특수문자 제거, 소문자로 변경
          txt = "".join(v for v in line if v not in string.punctuation).lower()
          txt = txt.split("\t")
          # 길이가 10이하인 문장만 사용
          engtxt = txt[0]; kortxt = txt[1]
          if len(engtxt) <=10 and len(kortxt) <=10 :
            self.eng_corpus.append(engtxt)
            self.kor_corpus.append(kortxt)
        except Exception as e:
          print(e, txt)

    self.engBOW = get_BOW(self.eng_corpus)
    self.korBOW = get_BOW(self.kor_corpus)
  def __len__(self):
    return len(self.engBOW)

  # 문장을 단어로 분리하고 마지막에 <EOS> 추가
  def get_seq(self, line):
    seq = line.split()
    seq.append("<EOS>")
    return seq

  def __getitem__(self, index):
    # 문자열을 숫자로 변환
    # 영어, 한국어
    data = np.array([ self.engBOW[txt] for txt in self.get_seq(self.eng_corpus[index]) ])
    label = np.array([ self.korBOW[txt] for txt in self.get_seq(self.kor_corpus[index]) ])
    return data,label

In [9]:
dataset = Eng2Kor()
data,label = next(iter(dataset))
data, label

list index out of range ['']


(array([2, 1]), array([2, 1]))

In [10]:
def loader(dataset):
  for i in range(len(dataset)):
    data,label = dataset[i]
    yield torch.tensor(data), torch.tensor(label)

In [11]:
for data, label in loader(dataset):
  print(data,label)

tensor([2, 1]) tensor([2, 1])
tensor([3, 1]) tensor([3, 1])
tensor([4, 1]) tensor([4, 1])
tensor([4, 1]) tensor([4, 1])
tensor([5, 1]) tensor([5, 1])
tensor([6, 1]) tensor([6, 1])
tensor([7, 1]) tensor([7, 1])
tensor([8, 1]) tensor([8, 1])
tensor([9, 1]) tensor([9, 1])
tensor([10,  1]) tensor([10,  1])
tensor([11,  1]) tensor([11,  1])
tensor([11,  1]) tensor([12,  1])
tensor([12,  1]) tensor([13, 14,  1])
tensor([13,  1]) tensor([15,  1])
tensor([13,  1]) tensor([16,  1])
tensor([13,  1]) tensor([15,  1])
tensor([14,  1]) tensor([17,  1])
tensor([15,  1]) tensor([3, 1])
tensor([15,  1]) tensor([3, 1])
tensor([16, 17,  1]) tensor([18,  1])
tensor([16, 18,  1]) tensor([19,  1])
tensor([16, 19,  1]) tensor([20, 21,  1])
tensor([20, 21,  1]) tensor([22, 23,  1])
tensor([22,  1]) tensor([24,  1])
tensor([23,  1]) tensor([8, 1])
tensor([24,  1]) tensor([25,  1])
tensor([25,  1]) tensor([26,  1])
tensor([25,  1]) tensor([27, 28, 29,  1])
tensor([25,  1]) tensor([30, 31,  1])
tensor([26,  1])

In [12]:
import torch.nn as nn
class Encoder(nn.Module):
  def __init__(self,input_size, hidden_size):
    super(Encoder,self).__init__()
    self.embeding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size,hidden_size)
  def forward(self, x, h):
    # 배치, 시계열 차원
    x = self.embeding(x).view(1,1,-1)
    output,hidden = self.gru(x,h)
    return output, hidden

In [13]:
class Decoder(nn.Module):
   def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=11):
       super(Decoder, self).__init__()

       # 임베딩층 정의
       self.embedding = nn.Embedding(output_size, hidden_size)

       # 어텐션 가중치를 계산하기 위한 MLP층
       self.attention = nn.Linear(hidden_size * 2, max_length)

       #특징 추출을 위한 MLP층
       self.context = nn.Linear(hidden_size * 2, hidden_size)

       # 과적합을 피하기 위한 드롭아웃 층
       self.dropout = nn.Dropout(dropout_p)

       # GRU층
       self.gru = nn.GRU(hidden_size, hidden_size)

       # 단어 분류를 위한 MLP층
       self.out = nn.Linear(hidden_size, output_size)

       # 활성화 함수
       self.relu = nn.ReLU()
       self.softmax = nn.LogSoftmax(dim=1)

   def forward(self, x, h, encoder_outputs):
       # 입력을 밀집 표현으로
       x = self.embedding(x).view(1, 1, -1)
       x = self.dropout(x)

       # 어텐션 가중치 계산
       attn_weights = self.softmax(
           self.attention(torch.cat((x[0], h[0]), -1)))

       # 어텐션 가중치와 인코더의 출력을 내적
       attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                encoder_outputs.unsqueeze(0))

       # 인코더 각 시점의 중요도와 민집표현을 합쳐
       # MLP층으로 특징 추출
       output = torch.cat((x[0], attn_applied[0]), 1)
       output = self.context(output).unsqueeze(0)
       output = self.relu(output)

       # GRU층으로 입력
       output, hidden = self.gru(output, h)

       # 예측된 단어 출력
       output = self.out(output[0])

       return output

학습

In [14]:
import random
import tqdm

from torch.optim.adam import Adam


# 학습에 사용할 프로세서 정의
device = "cuda" if torch.cuda.is_available() else "cpu"
# 학습에 사용할 데이터셋 정의
dataset = Eng2Kor()


# 인코더 디코더 정의
encoder = Encoder(input_size=len(dataset.engBOW), hidden_size=64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p=0.1).to(device)
# 인코더 디코더 학습을 위한 최적화 정의
encoder_optimizer = Adam(encoder.parameters(), lr=0.0001)
decoder_optimizer = Adam(decoder.parameters(), lr=0.0001)

list index out of range ['']


In [15]:
for epoch in range(50):
   iterator = tqdm.tqdm(loader(dataset), total=len(dataset))
   total_loss = 0

   for data, label in iterator:
       data = torch.tensor(data, dtype=torch.long).to(device)
       label = torch.tensor(label, dtype=torch.long).to(device)

       # 인코더의 초기 은닉 상태
       encoder_hidden = torch.zeros(1, 1, 64).to(device)
       # 인코더의 모든 시점의 출력을 저장하는 변수
       encoder_outputs = torch.zeros(11, 64).to(device)

       encoder_optimizer.zero_grad()
       decoder_optimizer.zero_grad()

       loss = 0
       for ei in range(len(data)):
           # 한 단어씩 인코더에 넣어줌
           encoder_output, encoder_hidden = encoder(
               data[ei], encoder_hidden)
           # 인코더의 은닉 상태를 저장
           encoder_outputs[ei] = encoder_output[0, 0]

       decoder_input = torch.tensor([[0]]).to(device)

       # 인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로 저장
       decoder_hidden = encoder_hidden
       # 티치포싱..... 디코더가 학습이 안되면 시간이 오래걸려서 강제로
       # 정답을 50%의 비율로 넣어준다
       use_teacher_forcing = True if random.random() < 0.5 else False

       if use_teacher_forcing:
           for di in range(len(label)):
               decoder_output = decoder(
                   decoder_input, decoder_hidden, encoder_outputs)

               # 직접적으로 정답을 다음 시점의 입력으로 넣어줌
               target = torch.tensor(label[di], dtype=torch.long).to(device)
               target = torch.unsqueeze(target, dim=0).to(device)
               loss += nn.CrossEntropyLoss()(decoder_output, target)
               decoder_input = target
       else:
           for di in range(len(label)):
               decoder_output = decoder(
                   decoder_input, decoder_hidden, encoder_outputs)

               # 가장 높은 확률을 갖는 단어의 인덱스가 topi
               topv, topi = decoder_output.topk(1)
               decoder_input = topi.squeeze().detach()

               # 디코더의 예측값을 다음 시점의 입력으로 넣어줌
               target = torch.tensor(label[di], dtype=torch.long).to(device)
               target = torch.unsqueeze(target, dim=0).to(device)
               loss += nn.CrossEntropyLoss()(decoder_output, target)

               if decoder_input.item() == 1:  # <EOS> 토큰을 만나면 중지
                   break
       # 전체 손실 계산
       total_loss += loss.item()/len(dataset)
       iterator.set_description(f"epoch:{epoch+1} loss:{total_loss}")
       loss.backward()

       encoder_optimizer.step()
       decoder_optimizer.step()

torch.save(encoder.state_dict(), "attn_enc.pth")
torch.save(decoder.state_dict(), "attn_dec.pth")

  0%|          | 0/304 [00:00<?, ?it/s]<ipython-input-15-dcdc4ec7f0e5>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.long).to(device)
<ipython-input-15-dcdc4ec7f0e5>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long).to(device)
<ipython-input-15-dcdc4ec7f0e5>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(label[di], dtype=torch.long).to(device)
epoch:1 loss:0.04131637748919035:   0%|          | 1/304 [00:01<06:42,  1.33s/it]<ipyth

성능평가

In [16]:
# 인코더 가중치 불러오기
encoder.load_state_dict(torch.load("attn_enc.pth", map_location=device))
# 디코더 가중치 불러오기
decoder.load_state_dict(torch.load("attn_dec.pth", map_location=device))


# 불러올 영어 문장을 랜덤하게 지정
idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
# 신경망이 번역한 문장
pred_sentence = ""

data, label = dataset[idx]
data = torch.tensor(data, dtype=torch.long).to(device)
label = torch.tensor(label, dtype=torch.long).to(device)

# 인코더의 초기 은닉 상태 정의
encoder_hidden = torch.zeros(1, 1, 64).to(device)
# 인코더 출력을 담기위한 변수
encoder_outputs = torch.zeros(11, 64).to(device)

인코더

In [17]:
for ei in range(len(data)):
   # 한 단어씩 인코더에 넣어줌
   encoder_output, encoder_hidden = encoder(
       data[ei], encoder_hidden)

   # 인코더의 출력을 저장
   encoder_outputs[ei] = encoder_output[0, 0]


# 디코더의 초기 입력
# 0은 <SOS>토큰
decoder_input = torch.tensor([[0]]).to(device)

# 인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로
decoder_hidden = encoder_hidden

디코더

In [18]:
for di in range(11):
    # 가장 높은 확률을 갖는 단어의 요소를 구함
   decoder_output = decoder(
                       decoder_input, decoder_hidden, encoder_outputs)
   topv, topi = decoder_output.topk(1)
   decoder_input = topi.squeeze().detach()

   # <EOS> 토큰을 만나면 중지
   if decoder_input.item() == 1:
       break

   # 가장 높은 단어를 문자열에 추가
   pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "

print(input_sentence)  # 영어 문장
print(pred_sentence)  # 한글 문장

come here
여기로 와 
